OK - vi skal have preprocesset det her data vi indsamler....



In [36]:
# import libraries
import pandas as pd
from pathlib import Path


In [64]:
# Get file list
fp = Path("../Lars/clean_data/")
files = list(fp.glob("*"))
files

[WindowsPath('../Lars/clean_data/processed_data_sales_1992_2022_161.csv'),
 WindowsPath('../Lars/clean_data/processed_data_sales_1992_2022_165.csv')]

In [72]:
# der er ingen grund til at tro at kolonnerne konsekvent hedder hvad de burde hedde:
column_mapping = {
    "municipalities": "muni_code",
    "municipality": "muni_code",
    "muni_code": "muni_code",
    "municode": "muni_code",
    "square meters_price":"square_meters_price"
}

def aar(str):
    return str.split('-')[0]
test = pd.read_csv(files[0])

In [73]:
# Loop igennem hver DataFrame og omdøb kolonnerne, hvis de findes. trækker year ud af datoen 
test = test\
    .rename(columns=column_mapping, inplace=False)\
    .assign(year = test["date_of_sale"].apply(lambda row: aar(row)))


,muni_code,housing_type,address,zip_code,town,purchase_amount,date_of_sale,type_of_sale,square meters,square_meters_price,rooms,year_of_construction,The percentage difference between the latest bid price and the sale price,year
0,161,Ejerlejlighed,"Dalvangsvej 13, 1. 3",2600,Glostrup,1000000,2022-12-28,Fam. Salg,88,11364,3,1967,NaN,2022
1,161,Ejerlejlighed,"Kochsvej 5, st",2600,Glostrup,600000,2022-12-28,Fam. Salg,99,6061,3,1906,NaN,2022
2,161,Ejerlejlighed,"Kochsvej 5, 1",2600,Glostrup,600000,2022-12-28,Fam. Salg,73,8219,3,1906,NaN,2022
3,161,Rækkehus,Anemonevej 27,2600,Glostrup,3800000,2022-12-22,Alm. Salg,136,27941,4,1974,-5%,2022
4,161,Villa,Statenevej 17,2600,Glostrup,2900000,2022-12-22,Alm. Salg,121,23967,4,1979,-9%,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,161,Villa,Sofielundsvej 5,2600,Glostrup,585550,1992-01-05,Alm. Salg,129,4539,6,1933,NaN,1992
9203,161,Villa,Østerager 25,2600,Glostrup,1280000,1992-01-04,Alm. Salg,160,8000,5,1987,NaN,1992
9204,161,Villa,Ejby Mosevej 43,2600,Glostrup,880000,1992-01-04,Alm. Salg,132,6667,4,1972,NaN,1992
9205,161,Villa,Solsortevej 27,2600,Glostrup,624754,1992-01-03,Alm. Salg,83,7527,3,1954,NaN,1992


Hvad vil vi så gruppere de her data på? Det er der ingen der ved. Men vi må hellere gøre noget, så vi kan komme videre...

Hvad vil vi gruppere på? Det ved vi ikke helt. Vi skal have grupperet på muni_code. Ville vi også have på housing type.
Det vil vi nok.Vi skal have trukket et år ud, for det er der ikke. Antal værelser? Type of sale? 

Uanset hvad, så skal den spytte en dataframe ud vi kan gøre noget videre med. og den skal gemmes til  fil.


In [89]:
group_vars = ["muni_code", "housing_type", "year"]
agg_vars = ["square_meters_price"]

test = test\
    .groupby(group_vars)[agg_vars].agg(["mean", "count"])\
    .reset_index()




Vi har nogen fæle outliers. Det kunne vi gøre noget ved på denne måde.
test["kantil"] = test.groupby(group_vars)[agg_vars].transform(lambda x: x.quantile(0.8))



#test[test['square_meters_price'] <= test['kantil']].groupby("year")["square_meters_price"].hist()

In [107]:
#af med multi index!
test.columns = range(test.shape[1])
test.columns = [0,1,"muni_code", "housing_type", "year", "avg_sqm_price", "count"]
test = test[["muni_code", "housing_type", "year", "avg_sqm_price", "count"]]


,muni_code,housing_type,year,avg_sqm_price,count
0,161,Ejerlejlighed,1992,10980.418803,117
1,161,Ejerlejlighed,1993,71789.604938,162
2,161,Ejerlejlighed,1994,6174.088235,68
3,161,Ejerlejlighed,1995,42871.463855,166
4,161,Ejerlejlighed,1996,9103.950000,100
...,...,...,...,...,...
88,161,Villa,2018,22993.475524,143
89,161,Villa,2019,23719.652174,138
90,161,Villa,2020,26677.388158,152
91,161,Villa,2021,29876.473684,133


In [108]:
test.to_csv("fittest.csv")